In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from tqdm import tqdm

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/OutputRecommendation1.txt')
df.head()

In [ ]:
dfDVD = df.loc[ df['group'] == 'DVD']
dfDVD

In [ ]:
type(dfDVD.iloc[0]['salesrank'])

In [ ]:
newDf = pd.DataFrame(columns=['ASINs', 'TitleSimilarity', 'CategorySimilarity', 'SalesRank', 'CoPurchasedProducts'])
newDf

In [ ]:
for i in range(int(dfDVD.shape[0]/4)):
    row = dfDVD.iloc[i][:]
    rowASIN = row['ASIN']
    cat_i = row['categories'].split(' ')
    title_i = row['title'].split(' ')
    #copurchasedASIN = row['copurchased'].split(' ')
    for coASIN in dfDVD.iloc[i]['copurchased'].split(' '):
        cat_jaccard_similarity = 0
        title_jaccard_similarity = 0
        cat_j_row = dfDVD.loc[ dfDVD['ASIN'] == coASIN ]
        cat_j = cat_j_row['categories'].to_string().split(' ')[1:]
#         if i<2:
#             print(coASIN)
#             print(cat_j)
        cat_iUcat_j = set(cat_i).union(set(cat_j))
        cat_iIcat_j = set(cat_i).intersection(set(cat_j))
        if len(cat_iUcat_j)>0:
            cat_jaccard_similarity = round( len(cat_iIcat_j)/len(cat_iUcat_j) , 2)
        else:
            cat_jaccard_similarity = 0
        
        title_j = cat_j_row['title'].to_string().split(' ')
        title_iUtitle_j = set(title_i).union(set(title_j))
        title_iItitle_j = set(title_i).intersection(set(title_j))
        if len(title_iUtitle_j)>0:
            title_jaccard_similarity = round( len(title_iItitle_j)/ len(title_iUtitle_j) , 2)
        else:
            title_jaccard_similarity = 0
        l = int(row['salesrank'])
        print(cat_j_row['salesrank'].to_string().split(' ')[0])
        try:
            r =  int(cat_j_row['salesrank'].to_string().split(' ')[0] )
            print(l,r)
            SalesRank = max( l, r)
            newDf = newDf.append(pd.Series([ rowASIN+' '+coASIN, title_jaccard_similarity, cat_jaccard_similarity, SalesRank, "Yes" ], index=newDf.columns ), ignore_index=True)
        except ValueError:
            pass
        

In [ ]:
newDf.head()

In [ ]:
newDf.shape

In [ ]:
dfBook = df.loc[ df['group'] == 'Book' ]
dfBook.head()

In [ ]:
for i in tqdm(range(int(dfBook.shape[0]/25))):
    bookASIN = dfBook.iloc[i]['ASIN']
    dvdASIN = newDf.iloc[i]['ASINs'].split(' ')[0]
    cat_i = dfBook.iloc[i]['categories'].split(' ')
    title_i = dfBook.iloc[i]['title'].split(' ')
    
    cat_jaccard_similarity = 0
    title_jaccard_similarity = 0
    #cat_j_row = dfDVD.loc[ dfDVD['ASIN'] == coASIN ]
    cat_j_row = dfBook.iloc[i][:]
    cat_j = cat_j_row['categories'].split(' ')[1:]

    cat_iUcat_j = set(cat_i).union(set(cat_j))
    cat_iIcat_j = set(cat_i).intersection(set(cat_j))
    if len(cat_iUcat_j)>0:
        cat_jaccard_similarity = round( len(cat_iIcat_j)/len(cat_iUcat_j) , 2)
    else:
        cat_jaccard_similarity = 0

    title_j = cat_j_row['title'].split(' ')
    title_iUtitle_j = set(title_i).union(set(title_j))
    title_iItitle_j = set(title_i).intersection(set(title_j))
    if len(title_iUtitle_j)>0:
        title_jaccard_similarity = round( len(title_iItitle_j)/ len(title_iUtitle_j) , 2)
    else:
        title_jaccard_similarity = 0
    l = int(row['salesrank'])
    #print(cat_j_row['salesrank'])
    try:
        r =  int(cat_j_row['salesrank'] )
        #print(l,r)
        SalesRank = max( l, r)
        newDf = newDf.append(pd.Series([ rowASIN+' '+coASIN, title_jaccard_similarity, cat_jaccard_similarity, SalesRank, "No" ], index=newDf.columns ), ignore_index=True)
    except ValueError:
        pass


In [ ]:
newDf.shape

In [ ]:
Y = newDf.pop('CoPurchasedProducts')
X = newDf
X.head()

In [ ]:
Y.head()

In [ ]:
X1 = newDf.pop('ASINs')

In [ ]:
X2 = newDf
X2.head()

In [ ]:
X2.shape

In [ ]:
Y_onehot = Y.copy()
Y_onehot = pd.get_dummies(Y_onehot, columns=['CoPurchasedProducts'], prefix ='CoPurchasedProducts')

print(Y_onehot.head())

In [ ]:
Y_onehot.pop('CoPurchasedProducts_Yes')
Y_onehot

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder(handle_unknown='ignore')
# enc.fit_transform(Y)
# #Y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, Y_onehot, test_size=0.5, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
# X_train.head()
y_train.head()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)
print(acc)

## Model2

In [ ]:
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression(tol=0.00001, solver='liblinear', random_state=42, C= 10, warm_start=True)

In [ ]:
model2.fit(X_train,y_train)

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
acc2 = accuracy_score(y_test,y_pred, normalize=True)
acc2

## Model3 (with CV)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
model3 = LogisticRegressionCV(Cs=2, tol=0.001, solver='lbfgs', random_state=42, cv=3, n_jobs=-1, max_iter=5000)

In [ ]:
model3.fit(X_train,y_train)

In [ ]:
y_pred = model3.predict(x_test)
acc3 = accuracy_score(y_test,y_pred, normalize=True)
acc3

In [ ]:
#To print the CV scores
model3.scores_

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier(random_state=42, max_depth=2, persort=True, criterion='gini', max_features = 'sqrt')

In [ ]:
clf1.fit(X_train,y_train)
y_pred = clf1.predict(x_test)
accuracy_score(y_test,y_pred,normalize=True)